# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

!pip install keras==2.9.0rc1
!pip install -q tensorflow_addons
# !pip install keras-cv-attention-models -q 

import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.9.0rc1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00


In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git

fatal: destination path 'finnegan-paleo-ml' already exists and is not an empty directory.


In [ ]:
# @title Data stored in drive
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
# dir_code = '/content/finnegan-paleo-ml/Code'
dir_code = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code' #Have to override here because data can't be version controlled and is in drive
# dir_git_home = '/content/finnegan-paleo-ml'
dir_git_home = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = dir_dataset_specs = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [ ]:
os.chdir(dir_code)
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor
os.chdir(dir_git_home)

In [ ]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [ ]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df = labels_df[labels_df['Broken'].isin(['unbroken', 'broken'])]

labels_df.head()


In [12]:
labels_df[(labels_df['Broken'] == 'micro')]

,,Broken,species,age,test,val
sample_name,object_num,,,,,


In [11]:
%cd /content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code

/content/drive/.shortcut-targets-by-id/1pfS62oxpwDOc9RME825AUzAZ_qVVMlxG/MV1012_SBB_images/ML_projects/Code


In [12]:
%cd /content/drive/MyDrive/MV1012_SBB_images/ML_projects/Processed_data/

/content/drive/.shortcut-targets-by-id/1pfS62oxpwDOc9RME825AUzAZ_qVVMlxG/MV1012_SBB_images/ML_projects/Processed_data


In [11]:
dir_ml_home

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects'

In [28]:
import shutil
for split in ['train', 'test', 'val']:
    # shutil.rmtree(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    os.mkdir(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    


In [23]:
git()

fatal: invalid reference: origin


In [ ]:
#Run data processing
data_processor.run_processing(dir_processed_data, dir_raw_training_images, labels_df, verbose=True, override=False)

## Create tensorflow dataset and push to google cloud

In [7]:
data_processor

<module 'data_processing.data_processor' from '/content/finnegan-paleo-ml/Code/data_processing/data_processor.py'>

In [13]:
import inspect
print(inspect.getsource(data_processor.create_cloud_dataset))

def create_cloud_dataset(dir_local_processed_data, dir_cloud_data, dir_dataset_specs, batch_size):
    # TODO: support for dataset data that's not in the same dir as specs
    cur_cwd = os.getcwd()
    os.chdir(dir_dataset_specs)
    import forams
    os.chdir(cur_cwd)
    builder_kwargs = {
        'image_label_dir': dir_local_processed_data,
    }
    training_set = tfds.load('forams', split='train', builder_kwargs=builder_kwargs)
    validation_set = tfds.load('forams', split='val', builder_kwargs=builder_kwargs)
    testing_set = tfds.load('forams', split='test', builder_kwargs=builder_kwargs)
    print(training_set)
    # os.chdir(cur_cwd)
    tf.data.experimental.save(training_set, f'{dir_cloud_data}/training{batch_size}')
    tf.data.experimental.save(validation_set, f'{dir_cloud_data}/validation{batch_size}')
    tf.data.experimental.save(testing_set, f'{dir_cloud_data}/testing{batch_size}')

    # Above code deprecated below not supported in colab default tf version
    # tf.d

In [9]:
dir_processed_data

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams'

In [14]:
data_processor.create_cloud_dataset(dir_processed_data, dir_google_cloud, dir_dataset_specs, 1024)

image_label_dir /content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams


Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/forams/1.0.0.incompleteQATHME/forams-train.tfrecord*...:   0%|          | …

Generating val examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/forams/1.0.0.incompleteQATHME/forams-test.tfrecord*...:   0%|          | 0…

Dataset forams downloaded and prepared to /root/tensorflow_datasets/forams/1.0.0. Subsequent calls will reuse this data.


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


image_label_dir /content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams
image_label_dir /content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams
<PrefetchDataset element_spec={'chamber_broken': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(416, 416, 3), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
